In [ ]:
from http import HTTPStatus
from urllib.parse import urlparse, unquote
from pathlib import PurePosixPath
import requests
from dashscope import ImageSynthesis
import os
import dashscope

# 以下为北京地域url，若使用新加坡地域的模型，需将url替换为：https://dashscope-intl.aliyuncs.com/api/v1
dashscope.base_http_api_url = 'https://dashscope.aliyuncs.com/api/v1'

prompt = "中国传统风格的彩色水墨画，描绘了一个名叫天虞的人，他有着独特的特征——反臂。他的手臂从肩膀处向后弯曲，手肘朝前，手掌则自然地放在背后，仿佛天生如此。天虞身穿古代服饰，长袍宽袖，衣袂飘飘，展现出一种超凡脱俗的气质。背景是山川河流，云雾缭绕，增添了几分神秘感"


# 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
# 新加坡和北京地域的API Key不同。获取API Key：https://help.aliyun.com/zh/model-studio/get-api-key
api_key = os.getenv("DASHSCOPE_API_KEY")

print('----sync call, please wait a moment----')
rsp = ImageSynthesis.call(api_key=api_key,
                          model="wan2.5-t2i-preview",
                          prompt=prompt,
                          n=1,
                          size='1024*1024')
print('response: %s' % rsp)
if rsp.status_code == HTTPStatus.OK:
    # 在当前目录下保存图片
    for result in rsp.output.results:
        file_name = PurePosixPath(unquote(urlparse(result.url).path)).parts[-1]
        with open('./%s' % file_name, 'wb+') as f:
            f.write(requests.get(result.url).content)
else:
    print('sync_call Failed, status_code: %s, code: %s, message: %s' %
          (rsp.status_code, rsp.code, rsp.message))


----sync call, please wait a moment----
response: {"status_code": 200, "request_id": "ba5bb891-3c09-4c2f-881b-b16ea819c7ad", "code": null, "message": "", "output": {"task_id": "1422247c-b9fa-40ce-a768-e5239b4e83cb", "task_status": "SUCCEEDED", "results": [{"url": "https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/d8/20251103/5af8a087/1422247c-b9fa-40ce-a768-e5239b4e83cb.png?Expires=1762255578&OSSAccessKeyId=LTAI5tKPD3TMqf2Lna1fASuh&Signature=N5%2BoFzEH3R6AG5RhCTtlutIDr4E%3D", "orig_prompt": "中国传统风格的彩色水墨画，描绘了一个名叫天虞的人，他有着独特的特征——反臂。他的手臂从肩膀处向后弯曲，手肘朝前，手掌则自然地放在背后，仿佛天生如此。天虞身穿古代服饰，长袍宽袖，衣袂飘飘，展现出一种超凡脱俗的气质。背景是山川河流，云雾缭绕，增添了几分神秘感", "actual_prompt": "中国传统彩色水墨画风格，描绘一位名为“天虞”的古代人物。天虞身形修长，面容清癯，神情淡然超逸，最显著的特征是其“反臂”——双臂自肩部向后自然弯曲，手肘朝前，手掌轻搭于背后，姿态奇异却毫无违和，仿佛天生如此。他身着宽袖长袍，衣料轻盈，衣袂随风飘动，呈现出流动的韵律感，整体服饰色调以青灰、月白与淡赭为主，辅以细腻的墨线勾勒纹样，彰显古雅脱俗之气。背景为层峦叠嶂的山川与蜿蜒流淌的江河，远山隐没于缭绕云雾之中，近处松石错落，水汽氤氲，营造出空灵神秘、仙气弥漫的意境。画面构图疏密有致，墨色浓淡相宜，色彩温润典雅，充分展现中国传统文人画的审美意趣与神话人物的独特气质。"}], "submit_time": "2025-11-03 19:26:06.133", "scheduled_time":

In [ ]:
import pandas as pd
from http import HTTPStatus
from urllib.parse import urlparse, unquote
from pathlib import Path, PurePosixPath
import requests
from dashscope import ImageSynthesis
import dashscope
from concurrent.futures import ThreadPoolExecutor
import os

dashscope.base_http_api_url = 'https://dashscope.aliyuncs.com/api/v1'
api_key = os.getenv("DASHSCOPE_API_KEY")

def generate_image(row, csv_path, images_base_dir):
    """为单行生成图片"""
    prompt = row['prompt']
    name = row['名字']  # 使用名字列

    # 构建保存路径: images/<一级目录>/<csv名>/<名字>.png
    csv_rel = Path(csv_path).relative_to('csv_by_category')
    parts = csv_rel.parts
    level1 = parts[0] if len(parts) >= 2 else None   # 一级目录（若有）
    csv_stem = Path(csv_path).stem                   # csv 文件名（去掉后缀）
    if level1:
        save_dir = images_base_dir / level1 / csv_stem
    else:
        save_dir = images_base_dir / csv_stem
    save_dir.mkdir(parents=True, exist_ok=True)

    # 使用名字作为文件名
    image_filename = f"{name}.png"
    image_path = save_dir / image_filename
    relative_image_path = str(image_path.relative_to(images_base_dir))

    # 跳过已存在的图片
    if image_path.exists():
        print(f'跳过已存在的图片: {relative_image_path}')
        return relative_image_path

    # 调用API生成图片
    print(f'正在生成: {relative_image_path}')
    rsp = ImageSynthesis.call(
        api_key=api_key,
        model="wan2.5-t2i-preview",
        prompt=prompt,
        n=1,
        size='1024*1024'
    )
    try:
        if rsp.status_code == HTTPStatus.OK:
            result = rsp.output.results[0]
            with open(image_path, 'wb') as f:
                f.write(requests.get(result.url).content)
            print(f'完成: {relative_image_path}')
            return relative_image_path
        else:
            print(f'生成失败: {rsp.status_code}')
            return None
    except Exception as e:
        print(f'处理异常: {e}')
        print(rsp)
        return None

def process_csv_file(csv_path, images_base_dir):
    """处理单个CSV文件"""
    df = pd.read_csv(csv_path)
    
    # 添加image_path列
    if 'image_path' not in df.columns:
        df['image_path'] = None
    
    # 使用线程池处理每一行
    try:
        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = []
            for idx, row in df.iterrows():
                future = executor.submit(generate_image, row, csv_path, images_base_dir)
                futures.append((idx, future))
    except Exception as e:
        print(f'线程池处理异常: {e}')
        # 收集结果
        for idx, future in futures:
            result = future.result()
            df.at[idx, 'image_path'] = result
    
    # 保存更新后的CSV
    df.to_csv(csv_path, index=False)
    print(f'已更新CSV: {csv_path}')

def main():
    csv_dir = Path('csv_by_category')
    images_dir = Path('images')
    images_dir.mkdir(exist_ok=True)
    
    # 遍历所有CSV文件
    csv_files = list(csv_dir.rglob('*.csv'))
    
    for csv_file in csv_files:
        print(f'\n处理文件: {csv_file}')
        process_csv_file(csv_file, images_dir)
    
    print('\n所有文件处理完成！')

if __name__ == '__main__':
    main()


处理文件: csv_by_category\乐舞\乐名.csv
跳过已存在的图片: 乐舞\乐名\九辯.png
跳过已存在的图片: 乐舞\乐名\九歌.png
跳过已存在的图片: 乐舞\乐名\九招.png
跳过已存在的图片: 乐舞\乐名\樂風.png
已更新CSV: csv_by_category\乐舞\乐名.csv

处理文件: csv_by_category\乐舞\乐器.csv
跳过已存在的图片: 乐舞\乐器\梧.png
跳过已存在的图片: 乐舞\乐器\鍾.png
跳过已存在的图片: 乐舞\乐器\磬.png
跳过已存在的图片: 乐舞\乐器\琴.png
跳过已存在的图片: 乐舞\乐器\瑟.png
已更新CSV: csv_by_category\乐舞\乐器.csv

处理文件: csv_by_category\乐舞\舞名.csv
跳过已存在的图片: 乐舞\舞名\儛.png
跳过已存在的图片: 乐舞\舞名\二人儛.png
跳过已存在的图片: 乐舞\舞名\干儛.png
跳过已存在的图片: 乐舞\舞名\九代.png
跳过已存在的图片: 乐舞\舞名\璆冕舞.png
已更新CSV: csv_by_category\乐舞\舞名.csv

处理文件: csv_by_category\人物\人名.csv
跳过已存在的图片: 人物\人名\天虞.png
跳过已存在的图片: 人物\人名\聽訞.png
跳过已存在的图片: 人物\人名\太子長琴.png
跳过已存在的图片: 人物\人名\危.png
跳过已存在的图片: 人物\人名\巫抵.png
跳过已存在的图片: 人物\人名\（鬼國）物.png
跳过已存在的图片: 人物\人名\巫.png
跳过已存在的图片: 人物\人名\巫抵.png
跳过已存在的图片: 人物\人名\巫臷民.png
跳过已存在的图片: 人物\人名\巫凡.png
跳过已存在的图片: 人物\人名\呉回.png
跳过已存在的图片: 人物\人名\巫姑.png
跳过已存在的图片: 人物\人名\無骨.png
跳过已存在的图片: 人物\人名\巫即.png
跳过已存在的图片: 人物\人名\王亥.png
跳过已存在的图片: 人物\人名\巫禮.png
跳过已存在的图片: 人物\人名\巫履.png
跳过已存在的图片: 人物\人名\巫羅.png
跳过已存在的图片: 人物\人名\沃民.png
跳过已存在的图

In [2]:
#把对应的image相对路径同步到csv里面

from pathlib import Path
import pandas as pd

def expected_rel_path(csv_path, name):
    # 目标结构: images/<一级目录>/<csv名>/<名字>.png 的相对 images 路径
    csv_rel = Path(csv_path).relative_to('csv_by_category')
    parts = csv_rel.parts
    level1 = parts[0] if len(parts) >= 2 else None
    csv_stem = Path(csv_path).stem
    return (Path(level1) / csv_stem / f"{name}.png") if level1 else (Path(csv_stem) / f"{name}.png")

def sync_one_csv(csv_path, images_base=Path('images')):
    df = pd.read_csv(csv_path)
    def compute(row):
        rel = expected_rel_path(csv_path, row['名字'])
        return str(rel) if (images_base / rel).exists() else None
    df['image_path'] = df.apply(compute, axis=1)
    df.to_csv(csv_path, index=False)
    print(f'已更新CSV: {csv_path}')

def main():
    for csv_file in Path('csv_by_category').rglob('*.csv'):
        sync_one_csv(csv_file)

if __name__ == '__main__':
    main()

已更新CSV: csv_by_category\乐舞\乐名.csv
已更新CSV: csv_by_category\乐舞\乐器.csv
已更新CSV: csv_by_category\乐舞\舞名.csv
已更新CSV: csv_by_category\人物\人名.csv
已更新CSV: csv_by_category\人物\尸.csv
已更新CSV: csv_by_category\人物\神名.csv
已更新CSV: csv_by_category\其他\其他.csv
已更新CSV: csv_by_category\其他\视肉.csv
已更新CSV: csv_by_category\动物\兽名.csv
已更新CSV: csv_by_category\动物\虫名.csv
已更新CSV: csv_by_category\动物\螺.csv
已更新CSV: csv_by_category\动物\贝.csv
已更新CSV: csv_by_category\动物\鱼名.csv
已更新CSV: csv_by_category\动物\鸟名.csv
已更新CSV: csv_by_category\动物\龟鳖.csv
已更新CSV: csv_by_category\器物\兵器.csv
已更新CSV: csv_by_category\器物\刑具.csv
已更新CSV: csv_by_category\器物\器物.csv
已更新CSV: csv_by_category\器物\服饰.csv
已更新CSV: csv_by_category\地名\山名.csv
已更新CSV: csv_by_category\地名\山系.csv
已更新CSV: csv_by_category\地名\建筑.csv
已更新CSV: csv_by_category\地名\森林.csv
已更新CSV: csv_by_category\地名\水名.csv
已更新CSV: csv_by_category\地名\泽名.csv
已更新CSV: csv_by_category\地名\洞穴.csv
已更新CSV: csv_by_category\地名\海名.csv
已更新CSV: csv_by_category\地名\渊名.csv
已更新CSV: csv_by_category\地名\荒漠.csv
已更新CSV: csv_by_ca